In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from numpy import *
import statsmodels.api as sm
from numpy.linalg import inv
from linearmodels import PanelOLS
from statsmodels.iolib.summary2 import summary_col
from numpy.linalg import inv
from collections import OrderedDict
from linearmodels.panel import compare
from linearmodels.panel import PooledOLS

import os
#cwd = os.getcwd()

# Problem 1

The dataset "full merit aid" contains information on college enrollment by year and by state along with merit and demographics such as gender and race. We are interested in looking at the effect of merit, demographics, year and state fixed effects on college enrollment i.e. 
\begin{equation}
college_{it} = \gamma_t + \alpha_i +\beta_1 merit_{it} + \beta_2 male_{it} + \beta_3 black_{it} + \beta_4 asian_{it} +  \epsilon_{it}
\end{equation}

##  Standard regression using all data

In [15]:
df = pd.read_csv("fullmeritaiddata.csv")
year = pd.Categorical(df['year'])
state = pd.Categorical(df['state'])
df = df.set_index(['state','year'])
df['year'] = year
df['state'] = state

Estimation using canned package for standard regression for panel data. The estimation uses all of the variables as well as dummy variables for year and state. The standard errors are clustered by state and year.

In [22]:
exog = ['merit','male','black','asian','year','state']
x = sm.add_constant(df[exog])
clust_state_year = PanelOLS(df['coll'], x).fit(cov_type='clustered',
                cluster_entity=True, cluster_time=True,use_lsdv=True)
parameters = clust_state_year.params
se_sy = clust_state_year.std_errors
print("Estimated parameters for exogenous variables \
from standard regression with state and year FE")
print('constant:',np.round(parameters[0],3))
print('merit:',np.round(parameters[1],3))
print('male:',np.round(parameters[2],3))
print('black:',np.round(parameters[3],3))
print('asian:',np.round(parameters[4],3))

Estimated parameters for exogenous variables from standard regression with state and year FE
constant: 0.442
merit: 0.034
male: -0.079
black: -0.15
asian: 0.169


In [23]:
print("Standard errors clustered by state and year \
for exogenous variables")
print('constant:',np.round(se_sy[0],3))
print('merit:',np.round(se_sy[1],3))
print('male:',np.round(se_sy[2],3))
print('black:',np.round(se_sy[3],3))
print('asian:',np.round(se_sy[4],3))

Standard errors clustered by state and year for exogenous variables
constant: 0.009
merit: 0.012
male: 0.008
black: 0.014
asian: 0.028


Panel regression with standard errors clusterd by state only

In [24]:
clust_state = PanelOLS(df.coll, x).fit(cov_type='clustered',
                                cluster_entity=True,use_lsdv=True)
se_s = clust_state.std_errors
print("Standard errors clustered by state for exogenous variables")
print('constant:',np.round(se_s[0],3))
print('merit:',np.round(se_s[1],3))
print('male:',np.round(se_s[2],3))
print('black:',np.round(se_s[3],3))
print('asian:',np.round(se_s[4],3))

Standard errors clustered by state for exogenous variables
constant: 0.009
merit: 0.013
male: 0.007
black: 0.014
asian: 0.03


Robust standard errors

In [25]:
robust = PanelOLS(df.coll, x).fit(cov_type='robust',use_lsdv=True)
se_rob = robust.std_errors
print("Robust standard errors for exogenous variables")
print('constant:',np.round(se_rob[0],3))
print('merit:',np.round(se_rob[1],3))
print('male:',np.round(se_rob[2],3))
print('black:',np.round(se_rob[3],3))
print('asian:',np.round(se_rob[4],3))

Robust standard errors for exogenous variables
constant: 0.025
merit: 0.015
male: 0.005
black: 0.007
asian: 0.013


In [ ]:
# Manual estimation of beta
df = pd.read_csv("fullmeritaiddata.csv")
state_num = sorted(df['state'].unique().tolist())
year_seq = sorted(df['year'].unique().tolist())
s_dummy = [] 
i = 0
while i<len(state_num):
    s_dummy.append(np.where(df['state'] == state_num[i], 1, 0))
    i += 1
    if i > len(state_num):
        break
s_dummy = pd.DataFrame(np.asmatrix(s_dummy).T,columns =state_num)

y_dummy = [] 
i = 0
while i<len(year_seq):
    y_dummy.append(np.where(df['year'] == year_seq[i], 1, 0))
    i += 1
    if i > len(year_seq):
        break
y_dummy = pd.DataFrame(np.asmatrix(y_dummy).T,columns =year_seq) 

In [ ]:
state_dummy = s_dummy.drop(11,axis=1)
year_dummy = y_dummy.drop(1989, axis=1)
new_df = pd.concat([df,state_dummy,year_dummy],axis=1)
y = np.asmatrix(df['coll']).T
exog_vars = new_df.drop(['coll','chst','year','state'],axis=1)
x = sm.add_constant(exog_vars)
x = np.asmatrix(x)
b = inv(x.T*x)*x.T*y
print('The estimated parameters')
print('constant:',np.round(np.asscalar(b[0,]),3))
print('merit:',np.round(np.asscalar(b[1,]),3))
print('male:',np.round(np.asscalar(b[2,]),3))
print('black:',np.round(np.asscalar(b[3,]),3))
print('asian:',np.round(np.asscalar(b[4,]),3))

In [ ]:
# Manual estimation of the robust standard errors
e = y - x*b
sigma2 = np.asscalar(np.divide((e.T*e),(len(y)-len(b))))
vcov = sigma2*inv(x.T*x)
stderr = sqrt(diag(vcov))
e2 = np.power(e,2)

In [ ]:
S = np.zeros(shape=(66,66))
for i in range(len(e2)):
    e_row = np.asscalar(e2[i])
    x_row = x[i,]
    product = e_row * x_row.T*x_row
    S += product
whcov = inv(x.T*x)*S*inv(x.T*x)
robust_se = sqrt(diag(whcov))
print('Robust standard errors')
print('constant:',np.asscalar(robust_se[0,]))
print('merit:',np.asscalar(robust_se[1,]))
print('male:',np.asscalar(robust_se[2,]))
print('black:',np.asscalar(robust_se[3,]))
print('asian:',np.asscalar(robust_se[4,]))

## Standard regression using all data but weighted so that each state in each year gets the same weight

Let the weight for state $i$ at year $j$ equal $w_{ij}$ so letting each state in each year get the same weight implies
\begin{equation}
w_{11,1993} = w_{12,1993} ... = w_{95,1993} \\
w_{11,2000} = w_{12,2000}... = w_{95,2000}\\
...
\end{equation}

In [11]:
df = pd.read_csv("fullmeritaiddata.csv")
year = pd.Categorical(df['year'])
state = pd.Categorical(df['state'])
df = df.set_index(['state','year'])
df['year'] = year
df['state'] = state
state_num = sorted(df['state'].unique().tolist())
year_seq = sorted(df['year'].unique().tolist())

In [12]:
denom = []
for j in year_seq:
    d = len(df[df['year']==j])
    denom.append(d)
weights_t = []
for i in range(len(denom)):
    e = 1 / denom[i]
    weights_t.append(e)
#weights_t

In [17]:
w_df = pd.DataFrame()
for i in range(len(year_seq)):
    ydf = df[df['year'] == year_seq[i]].astype(float)
    wdf = weights_t[i]*ydf[['coll','merit','male','black','asian']]
    w_df = w_df.append(wdf)

In [18]:
weighted_df = w_df.reset_index(level=['state','year'])
year = pd.Categorical(weighted_df['year'])
state = pd.Categorical(weighted_df['state'])
weighted_df = weighted_df.set_index(['state','year'])
weighted_df['year'] = year
weighted_df['state'] = state

In [19]:
exog = ['merit','male','black','asian','state','year']
x = sm.add_constant(weighted_df[exog])
equal_w = PooledOLS(weighted_df.coll, x).fit\
(cov_type='clustered',cluster_entity=True)
param_w = equal_w.params
print("Estimated parameters from regression with equal \
weights for each state in each year")
print('constant:',np.round(param_w[0],3))
print('merit:',np.round(param_w[1],3))
print('male:',np.round(param_w[2],3))
print('black:',np.round(param_w[3],3))
print('asian:',np.round(param_w[4],3))

Estimated parameters from regression with equal weights for each state in each year
constant: 0.005
merit: 0.027
male: -0.083
black: -0.148
asian: 0.173


In [ ]:
# Manual estimation
weighted_df = w_df.reset_index(level=['state', 'year'])
s_dummy = [] 
i = 0
while i<len(state_num):
    s_dummy.append(np.where(weighted_df['state'] == state_num[i], 1, 0))
    i += 1
    if i > len(state_num):
        break
s_dummy = pd.DataFrame(np.asmatrix(s_dummy).T,columns =state_num)

y_dummy = [] 
i = 0
while i<len(year_seq):
    y_dummy.append(np.where(weighted_df['year'] == year_seq[i], 1, 0))
    i += 1
    if i > len(year_seq):
        break
y_dummy = pd.DataFrame(np.asmatrix(y_dummy).T,columns =year_seq) 

In [ ]:
state_dummy = s_dummy.drop(11,axis=1)
year_dummy = y_dummy.drop(1989, axis=1)
new_df = pd.concat([weighted_df,state_dummy,year_dummy],axis=1)
y = np.asmatrix(weighted_df['coll']).T
exog_vars = new_df.drop(['coll','year','state'],axis=1)
x = sm.add_constant(exog_vars)
x = np.asmatrix(x)
b = inv(x.T*x)*x.T*y
print('The estimated parameters')
print('constant:',np.asscalar(b[0,]))
print('merit:',np.asscalar(b[1,]))
print('male:',np.asscalar(b[2,]))
print('black:',np.asscalar(b[3,]))
print('asian:',np.asscalar(b[4,]))

## Regression on all data but with the mean of all variable by $state \times year$ with robust standard errors and clustered by state

In [7]:
df = pd.read_csv("fullmeritaiddata.csv")
state_num = sorted(df['state'].unique().tolist())
year_seq = sorted(df['year'].unique().tolist())
# coll_avg = df.groupby(['year','state'])['coll'].mean()
def columnavg(column):
    col_avg = []
    for i in state_num:
        data = column[df['state'] == i]
        for j in year_seq:
            col_data = data[df['year'] == j]
            avg = mean(col_data)
            col_avg.append(avg)
    return col_avg

In [8]:
coll_avg = np.asarray(columnavg(df['coll']))
merit_avg = np.asarray(columnavg(df['merit']))
male_avg = np.asarray(columnavg(df['male']))
black_avg = np.asarray(columnavg(df['black']))
asian_avg = np.asarray(columnavg(df['asian']))
year = np.asarray(np.tile(year_seq,51))
state = np.asarray(np.repeat(state_num,12))

In [9]:
mean = np.stack((coll_avg,merit_avg,male_avg,
        black_avg,asian_avg,year,state), axis=1)
meandf = pd.DataFrame(mean, columns = ['coll',
'merit','male','black','asian','year','state'])
year = pd.Categorical(meandf['year'])
state = pd.Categorical(meandf['state'])
meandf = meandf.set_index(['state','year'])
meandf['year'] = year
meandf['state'] = state

Estimation with clustered standard errors by state

In [56]:
exog = ['merit','male','black','asian','state','year']
x = sm.add_constant(meandf[exog])
mean_clust_state = PooledOLS(meandf.coll, x).fit\
(cov_type='clustered',cluster_entity=True)
parameters = mean_clust_state.params
se_ms = mean_clust_state.std_errors
print("Estimated parameters")
print('constant:',np.round(parameters[0],3))
print('merit:',np.round(parameters[1],3))
print('male:',np.round(parameters[2],3))
print('black:',np.round(parameters[3],3))
print('asian:',np.round(parameters[4],3))

Estimated parameters
constant: 0.473
merit: 0.05
male: -0.11
black: -0.168
asian: -0.008


In [131]:
print("Clustered standard errors by state")
print('constant:',se_ms[0])
print('merit:',se_ms[1])
print('male:',se_ms[2])
print('black:',se_ms[3])
print('asian:',se_ms[4])

Clustered standard errors by state
constant: 0.0006205634598635754
merit: 0.013582542845276444
male: 0.057504338030208234
black: 0.07345129878399519
asian: 0.14929280330387235


Estimation with robust standard errors

In [58]:
mean_robust = PooledOLS(meandf.coll, x).fit(cov_type='robust')
se_mr = mean_robust.std_errors
print("Robust standard errors for exogenous variables")
print('constant:',se_mr[0])
print('merit:',se_mr[1])
print('male:',se_mr[2])
print('black:',se_mr[3])
print('asian:',se_mr[4])

Robust standard errors for exogenous variables
constant: 0.044675795608873484
merit: 0.013790906367855025
male: 0.055264034758472295
black: 0.07524576550883985
asian: 0.19017266800840946


## Weighted regression of part c

Regression using the mean of all values by $state \times year$ but weighting each state in each year equally, so using the weights found in part b.

In [117]:
ind_df = pd.DataFrame()
for i in range(len(year_seq)):
    ydf = meandf[meandf['year'] == year_seq[i]].astype(float)
    wdf = weights_t[i]*ydf[['coll','merit','male','black','asian']]
    ind_df = ind_df.append(wdf)

In [118]:
mean_w_df = ind_df.reset_index(level=['state', 'year'])
year = pd.Categorical(mean_w_df['year'])
state = pd.Categorical(mean_w_df['state'])
mean_w_df = mean_w_df.set_index(['state','year'])
mean_w_df['year'] = year
mean_w_df['state'] = state

In [119]:
exog = ['merit','male','black','asian','state','year']
x = sm.add_constant(mean_w_df[exog])
mean_w = PooledOLS(mean_w_df.coll, x).fit(cov_type='robust')
parameters = mean_w.params
se_ms = mean_w.std_errors
print("Estimated parameters")
print('constant:',parameters[0])
print('merit:',parameters[1])
print('male:',parameters[2])
print('black:',parameters[3])
print('asian:',parameters[4])

Estimated parameters
constant: 0.005525033597786941
merit: 0.04310606428141104
male: -0.12114553746433597
black: -0.18097761958777
asian: 0.08600217844654579


In [120]:
print("Robust standard errors")
print('constant:',se_ms[0])
print('merit:',se_ms[1])
print('male:',se_ms[2])
print('black:',se_ms[3])
print('asian:',se_ms[4])

Robust standard errors
constant: 0.0006205634598635754
merit: 0.013582542845276444
male: 0.057504338030208234
black: 0.07345129878399519
asian: 0.14929280330387235


Regression using the mean of all values by $state \times year$ but using weights representative of the sample of individuals.

In [121]:
state_num = sorted(df['state'].unique().tolist())
year_seq = sorted(df['year'].unique().tolist())
weight_s = []
for i in year_seq:
    ydf = df[df['year'] == i].astype(float)
    for j in state_num:
        sdf = ydf[ydf['state'] == j].astype(float)
        wei = len(sdf) / len(ydf)
        weight_s.append(wei)

In [129]:
sorted_df = meandf.sort_values(by=['year','state']).astype(float)
weight_s = np.asmatrix(weight_s).T
weightedcol = np.multiply(weight_s.astype(float),sorted_df[['coll',
                                'merit','male','black','asian']])
col_weights = pd.DataFrame(weightedcol,columns =['coll','merit','male','black','asian'])
ys = sorted_df[['year','state']]
ys = ys.reset_index(drop=True)
df_weights = pd.concat([col_weights,ys],axis=1)

In [130]:
weight_meandf =  df_weights.set_index(['year','state'])
weight_meandf['year'] = year
weight_meandf['state'] = state
exog = ['merit','male','black','asian','state','year']
x = sm.add_constant(weight_meandf[exog])
ind_weight = PooledOLS(weight_meandf.coll, x).fit\
(cov_type='robust')
parameters = ind_weight.params
se = ind_weight.std_errors
print("Estimated parameters for exogenous variables \
from standard regression with state and year FE")
print('constant:',np.round(parameters[0],3))
print('merit:',np.round(parameters[1],3))
print('male:',np.round(parameters[2],3))
print('black:',np.round(parameters[3],3))
print('asian:',np.round(parameters[4],3))

Estimated parameters for exogenous variables from standard regression with state and year FE
constant: 0.002
merit: 0.048
male: 0.465
black: 0.224
asian: 0.831


In all different estimations, the sign for merit remain positive implying that receiving merit increases the likelihood of being enrolled in college. The lowest value for merit is under the specification that each state in each year has equal weights and the highest is the estimation with the mean of all variables. The estimation with mean of all variables and mean with representative weights are both higher than the standard regression estimation.

## Event study looking at the effects of the merit aid program before and after it was implemented.

In [133]:
df = pd.read_csv("fullmeritaiddata.csv")
year = pd.Categorical(df['year'])
state = pd.Categorical(df['state'])
df = df.set_index(['state','year'])
df['year'] = year
df['state'] = state

Finding the states that received treatment i.e. merit

In [179]:
meritdf = df[df['merit'] == 1]
meritdf = meritdf.sort_values(by=['year'])
merit_st = meritdf['state'].unique().tolist()
print("Merit states",merit_st)

Merit states [71, 58, 64, 59, 85, 72, 57, 61, 88, 34]


Finding the years that the treatment was implemented in the states that received the treatment

In [180]:
df = pd.read_csv("fullmeritaiddata.csv")
merit_year = []
for i in merit_st:
    merit_year.append(df[(df['merit'] == 1) & (df['state'] == i)]['year'].min())

merit_start = pd.DataFrame({'year': merit_year}, index=merit_st)
merit_start

,year
71,1991
58,1993
64,1996
59,1997
85,1997
72,1998
57,1998
61,1999
88,2000
34,2000


Finding the year right before the treatment year for the states that implemented the merit treatment

In [222]:
pre_merit_year = []
for i in merit_st:
    pre_merit_year.append(df[(df['merit'] == 0) & (df['state'] == i)]['year'].max())

pre_merit = pd.DataFrame({'year': pre_merit_year}, index=merit_st)
pre_merit

,year
71,1990
58,1992
64,1995
59,1996
85,1996
72,1997
57,1997
61,1998
88,1999
34,1999


Effect of merit can be estimated by taking the difference in means of $coll$. The first difference is between the merit states after implementing merit and before so I will take the mean of $coll$ for states that implemented merit before and after implementation and take the difference in the means.

In [228]:
post_df = []
for i in merit_st:
    post_df.append(df[(df['state']==i)&(df['merit']==1)&
    (df['year']==merit_start.loc[i,'year'])].groupby(['year'])['coll'].mean())

In [225]:
pre_df = []
for i in merit_st:
    pre_df.append(df[(df['state']==i)&(df['merit']==0)&
    (df['year']==pre_merit.loc[i,'year'])].groupby(['year'])['coll'].mean())

In [268]:
post_coll = []
for i in range(len(post_df)):
    post_coll.append(np.asscalar(np.asmatrix(post_df[i])))
    
pre_coll = []
for i in range(len(pre_df)):
    pre_coll.append(np.asscalar(np.asmatrix(pre_df[i])))

In [279]:
post_coll_df = pd.DataFrame(post_coll)
pre_coll_df = pd.DataFrame(pre_coll)
treat_df = pd.DataFrame({'Posttreat mean coll': np.round(post_coll, 4),
                           'Pretreat mean coll': np.round(pre_coll, 4)},
                          index=merit_st)

The differences in mean from the pretreateament and posttreatment is calculated

In [280]:
treat_df['diff'] = treat_df['Posttreat mean coll'] - treat_df['Pretreat mean coll']
treat_df

,Posttreat mean coll,Pretreat mean coll,diff
71,0.4000,0.2708,0.1292
58,0.3793,0.3556,0.0237
64,0.4773,0.3659,0.1114
59,0.4567,0.3361,0.1206
85,0.3200,0.3333,-0.0133
72,0.3889,0.3651,0.0238
57,0.5405,0.4857,0.0548
61,0.4211,0.4390,-0.0179
88,0.4074,0.3333,0.0741
34,0.4724,0.5154,-0.0430
